# Weight to binary

## Input

In [1]:
import numpy as np
QN = 6
QM = 11
def real_to_Qnm(real, n, m):
    if(real >= 0):
        return int(np.round(real*(2**m)).astype(int))
    else:
        return int(2**(n+m+1) + np.round(real*(2**m)).astype(int))
def Qnm_to_real(real, n, m):
    real = int(real) & int(2**(n+m+1)-1)
    if(real >= 2**(n+m)):
        return (real-2**(n+m+1))/(2**m)
    else:
        return real/(2**m)

In [2]:
#https://stackoverflow.com/questions/4838994/float-to-binary
#64bit IEE754
def float_to_bin(x):
  if x == 0:
    return "0" * 64
  w, sign = (float.hex(x), 0) if x > 0 else (float.hex(x)[1:], 1)
  mantissa, exp = int(w[4:17], 16), int(w[18:])
  return "{}{:011b}{:052b}".format(sign, exp + 1023, mantissa)


In [3]:
X_test=np.load("X_test.npy")
X_test=X_test[0:10]
X_test.shape[0]
X_test

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
        1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
        1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
        1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
        1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       

### dec

In [3]:
fin_Wz  = open('Input_X.dat', 'w')
cc=0
for j in range(X_test.shape[0]) :
    for i in range(X_test.shape[1]) :
        cc=cc+1
        fin_Wz.write("{0}".format(int(X_test[j,i])))
        if cc == 48:
            fin_Wz.write("\n")
            cc = 0
fin_Wz.close()

NameError: name 'X_test' is not defined

In [4]:
from tensorflow.keras.models import load_model

model_file_name='my_model_dc_parsec.h5'
model_ = load_model(model_file_name)
model_.layers

## Embedding

In [6]:
print("embedding:",model_.layers[0].get_weights()[0].shape)
print("embedding:",model_.layers[0].get_weights()[0])
embedding_weight=model_.layers[0].get_weights()[0]
embedding_INPUT_SZ=model_.layers[0].get_weights()[0].shape[0]
embedding_HIDDEN_SZ=model_.layers[0].get_weights()[0].shape[1]
print(embedding_INPUT_SZ,embedding_HIDDEN_SZ)

embedding: (2, 8)
embedding: [[ 0.1511789   0.21257462  0.02249864  0.05298556  0.3013374   0.04191065
   0.20380408  0.01847126]
 [-0.58739895 -0.8522611  -0.12063408 -0.9246319  -1.0010942  -0.925209
  -0.6365828  -0.16443335]]
2 8


In [8]:
fin_Wz  = open('weight_Em_fpb.det', 'w')
for j in range(embedding_INPUT_SZ) :
	for i in range(embedding_HIDDEN_SZ) :
		fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(embedding_weight[j,i],QN,QM))))
fin_Wz.close()

## LSTM

In [9]:
import numpy as np
print("lstm:",np.array(model_.layers[1].get_weights()).shape)
print("lstm_0:",model_.layers[1].get_weights()[0].shape)
print("lstm_1:",model_.layers[1].get_weights()[1].shape)
print("lstm_2:",model_.layers[1].get_weights()[2].shape)
print("total: 12200")

lstm: (3,)
lstm_0: (8, 128)
lstm_1: (32, 128)
lstm_2: (128,)
total: 12200


In [10]:
#print(model_.layers[1].trainable_weights)
units = int(int(model_.layers[1].trainable_weights[0].shape[1])/4)
print("No. units: ", units)

W = model_.layers[1].get_weights()[0]
R = model_.layers[1].get_weights()[1]
b = model_.layers[1].get_weights()[2]

W_i = W[:, :units]
W_f = W[:, units: units * 2]
W_c = W[:, units * 2: units * 3]
W_o = W[:, units * 3:]

R_i = R[:, :units]
R_f = R[:, units: units * 2]
R_c = R[:, units * 2: units * 3]
R_o = R[:, units * 3:]

b_i = b[:units]
b_f = b[units: units * 2]
b_c = b[units * 2: units * 3]
b_o = b[units * 3:]

No. units:  32


In [11]:
print(W_i.shape,R_i.shape,b_i.shape)
print(W_f.shape,R_f.shape,b_f.shape)
# https://stackoverflow.com/questions/42861460/how-to-interpret-weights-in-a-lstm-layer-in-keras

(8, 32) (32, 32) (32,)
(8, 32) (32, 32) (32,)


In [12]:
b_f

array([1.2852685 , 1.2954607 , 1.1132425 , 1.076543  , 1.2984583 ,
       1.137377  , 1.064606  , 1.2264651 , 1.3859788 , 1.1612115 ,
       1.2088773 , 1.2976155 , 1.1497114 , 1.5215207 , 1.0806828 ,
       0.9369431 , 1.036492  , 1.0942913 , 1.0145316 , 1.3832744 ,
       0.8848595 , 1.0030036 , 1.6137878 , 1.2144096 , 0.97843343,
       1.0646273 , 1.0207555 , 1.1481979 , 1.21238   , 1.370541  ,
       0.7913532 , 1.0583173 ], dtype=float32)

In [13]:
fin_Wz  = open('weight_W_i_fpb.dat', 'w')
for j in range(W_i.shape[0]) :
	for i in range(W_i.shape[1]) :
		fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(W_i[j,i],QN,QM))))
fin_Wz.close()

In [14]:
fin_Wz  = open('weight_W_f_fpb.dat', 'w')
for j in range(W_f.shape[0]) :
	for i in range(W_f.shape[1]) :
		fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(W_f[j,i],QN,QM))))
fin_Wz.close()

In [15]:
fin_Wz  = open('weight_W_c_fpb.dat', 'w')
for j in range(W_c.shape[0]) :
	for i in range(W_c.shape[1]) :
		fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(W_c[j,i],QN,QM))))
fin_Wz.close()

In [16]:
fin_Wz  = open('weight_W_o_fpb.dat', 'w')
for j in range(W_o.shape[0]) :
	for i in range(W_o.shape[1]) :
		fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(W_o[j,i],QN,QM))))
fin_Wz.close()

### R

In [17]:
fin_Wz  = open('weight_R_i_fpb.dat', 'w')
for j in range(R_i.shape[0]) :
	for i in range(R_i.shape[1]) :
		fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(R_i[j,i],QN,QM))))
fin_Wz.close()

In [18]:
fin_Wz  = open('weight_R_f_fpb.dat', 'w')
for j in range(R_f.shape[0]) :
	for i in range(R_f.shape[1]) :
		fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(R_f[j,i],QN,QM))))
fin_Wz.close()

In [19]:
fin_Wz  = open('weight_R_c_fpb.dat', 'w')
for j in range(R_c.shape[0]) :
	for i in range(R_c.shape[1]) :
		fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(R_c[j,i],QN,QM))))
fin_Wz.close()

In [20]:
fin_Wz  = open('weight_R_o_fpb.dat', 'w')
for j in range(R_o.shape[0]) :
	for i in range(R_o.shape[1]) :
		fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(R_o[j,i],QN,QM))))
fin_Wz.close()

## B

In [21]:
fin_Wz  = open('weight_b_i_fpb.dat', 'w')
for j in range(b_i.shape[0]) :
	fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(b_i[j],QN,QM))))
fin_Wz.close()

In [22]:
fin_Wz  = open('weight_b_f_fpb.dat', 'w')
for j in range(b_f.shape[0]) :
	fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(b_f[j],QN,QM))))
fin_Wz.close()

In [23]:
fin_Wz  = open('weight_b_c_fpb.dat', 'w')
for j in range(b_c.shape[0]) :
	fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(b_c[j],QN,QM))))
fin_Wz.close()

In [24]:
fin_Wz  = open('weight_b_o_fpb.dat', 'w')
for j in range(b_o.shape[0]) :
	fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(b_o[j],QN,QM))))
fin_Wz.close()

## Dense

In [29]:
print("dense_w:",model_.layers[3].get_weights()[0].shape)
print("dense_b:",model_.layers[3].get_weights()[1].shape)
dense_w=model_.layers[3].get_weights()[0]
dense_b=model_.layers[3].get_weights()[1]

dense_w: (32, 16)
dense_b: (16,)


In [31]:
dense_b

array([-1.8013229 , -1.0441449 , -0.49105257, -0.77370507, -0.4876282 ,
       -0.4249529 , -0.38504785, -0.34773955, -0.3683724 , -0.22437364,
       -0.33965763, -0.3289736 , -0.12665622,  0.01863566, -0.5537919 ,
        0.42437387], dtype=float32)

### W

In [32]:
fin_Wz  = open('weight_dense_w_fpb.dat', 'w')
for j in range(dense_w.shape[0]) :
	for i in range(dense_w.shape[1]) :
		fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(dense_w[j,i],QN,QM))))
fin_Wz.close()

### B

In [33]:
fin_Wz  = open('weight_dense_b_fpb.dat', 'w')
for j in range(dense_b.shape[0]) :
	fin_Wz.write("{0:018b}\n".format(int(real_to_Qnm(dense_b[j],QN,QM))))
fin_Wz.close()